In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
import pickle

In [2]:
data=pd.read_csv("D:\Genai-lc-hf\Datasets\Churn_Modelling.csv")

In [3]:
# here i am predicting the estimated salary since it is a continuous value thus making this as a regression problem

# encode the categorical vars
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [4]:
#one-hot encoder 'Geography'
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [5]:
#combining
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)


In [6]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,3,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,4,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,5,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
#splitting - Drop irrelevant columns that can't be scaled

# Drop RowNumber, CustomerId, Surname (non-numeric), EstimatedSalary (target), and Exited
X = data.drop(['RowNumber', 'CustomerId', 'Surname', 'EstimatedSalary', 'Exited'], axis=1)
y = data['EstimatedSalary']

In [8]:
## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)


ANN regression Problem Statement 

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense


In [11]:
#model building
model=Sequential ([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) #output layer for regression
    ])

#compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

d:\Genai-lc-hf\tfenv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
#setup tensor board

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [14]:
#setup early stopping 
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [15]:
#training

history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorflow_callback]  
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 100376.6328 - mae: 100376.6328 - val_loss: 98509.0781 - val_mae: 98509.0781
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 99585.4844 - mae: 99585.4844 - val_loss: 96916.3906 - val_mae: 96916.3906
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 96793.8750 - mae: 96793.8750 - val_loss: 92818.3359 - val_mae: 92818.3359
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 91231.4688 - mae: 91231.4688 - val_loss: 85896.7969 - val_mae: 85896.7969
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 83041.0312 - mae: 83041.0312 - val_loss: 76874.7109 - val_mae: 76874.7109
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 73484.2031 - mae: 73484.2031 - val_loss: 67616.1094 - val_mae: 67616.1094
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 64388.8789 - mae: 64388.8789 - val_loss: 59814.0742 - val_mae: 59814.0742
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step 

In [16]:
%load_ext tensorboard

In [17]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 4900), started 10:50:02 ago. (Use '!kill 4900' to kill it.)

In [18]:
#evaluate the model on the test data 

test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE:{test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50245.1055 - mae: 50245.1055
Test MAE:50245.10546875


In [19]:
model.save('regression_model.h5')